In [1]:
%cd ../..

/share/home2/olarinoyem/Project/Modern-Time-Series-Forecasting-with-Python-2E


In [2]:
import numpy as np
import os
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm
import statsmodels.api as sm
import warnings
import random
from IPython.display import display, HTML
np.random.seed(42)
random.seed(42)
tqdm.pandas()

/tmp/ipykernel_4168864/3033596466.py:5: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Reading and Selecting Households

In [3]:
try:
    lclid_acorn_map = pd.read_pickle("data/london_smart_meters/preprocessed/london_smart_meters_lclid_acorn_map.pkl")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 02 - Preprocessing London Smart Meter Dataset.ipynb in Chapter02
    </div>
    """))

In [4]:
affluent_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Affluent", ["LCLid",'file']]
adversity_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Adversity", ["LCLid",'file']]
comfortable_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Comfortable", ["LCLid",'file']]


Let's take a subset of the data because if we take everything, it will hit your RAM. Depending on how much RAM you have, we can choose larger data. But to maintain the variety in the dataset, we will do stratified sampling based on Acorn classifications.

* <= 50 households for 4GB RAM
* 50 - 100 households for 8GB RAM
* 100-150 households for 16GB RAM
* 250 households for 32GB RAM

Let's sample 150 households now, but feel free to reduce of increase as per your hardware constraints

150 households means 50 each from the three Acorn Groups - Affluent, Comfortable, Adversity (we are ignoring the households with unknown ACORN groups)

In [5]:
selected_households = pd.concat(
    [
        affluent_households.sample(100, random_state=76),
        comfortable_households.sample(100, random_state=76),
        adversity_households.sample(100, random_state=76),
    ]
)
selected_households['block']=selected_households.file.str.split("_", expand=True).iloc[:,1].astype(int)

In [6]:
# extracting the paths to the different blocks and extracting the starting and ending blocks
path_blocks = [
    (p, *list(map(int, p.name.split("_")[5].split(".")[0].split("-"))))
    for p in Path("data/london_smart_meters/preprocessed").glob(
        "london_smart_meters_merged_block*"
    )
]

In [7]:
household_df_l = []
for path, start_b, end_b in tqdm(path_blocks):
    block_df = pd.read_parquet(path)
    selected_households['block'].between
    mask = selected_households['block'].between(start_b, end_b)
    lclids = selected_households.loc[mask, "LCLid"]
    household_df_l.append(block_df.loc[block_df.LCLid.isin(lclids)])

100%|██████████| 14/14 [02:55<00:00, 12.52s/it]


In [8]:
block_df = pd.concat(household_df_l)
del household_df_l
block_df.head()

,LCLid,start_timestamp,frequency,energy_consumption,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
4073,MAC001820,2012-06-08,30min,"[0.426, 0.411, 0.337, 0.271, 0.253, 0.212, 0.2...",30240,Std,ACORN-J,Comfortable,block_72,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[199, 199, 196, 196, 196, 196, 196, 196, 205, ...","[13.66, 13.66, 13.54, 13.54, 13.15, 13.15, 13....","[9.54, 9.54, 9.89, 9.89, 10.77, 10.77, 10.78, ...","[994.85, 994.85, 994.46, 994.46, 994.2, 994.2,...","[13.66, 13.66, 13.54, 13.54, 13.15, 13.15, 13....","[8.24, 8.24, 8.27, 8.27, 8.48, 8.48, 8.54, 8.5...","[rain, rain, rain, rain, rain, rain, rain, rai...","[wind, wind, wind, wind, wind, wind, wind, win...","[0.76, 0.76, 0.79, 0.79, 0.85, 0.85, 0.86, 0.8...","[Breezy and Mostly Cloudy, Breezy and Mostly C..."
4105,MAC004689,2012-07-14,30min,"[0.0559999999999999, 0.013, 0.068, 0.05, 0.076...",28512,ToU,ACORN-J,Comfortable,block_72,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[236, 236, 239, 239, 244, 244, 244, 244, 248, ...","[14.29, 14.29, 14.22, 14.22, 14.19, 14.19, 13....","[13.11, 13.11, 13.13, 13.13, 13.09, 13.09, 12....","[1000.16, 1000.16, 1000.18, 1000.18, 1000.07, ...","[14.29, 14.29, 14.22, 14.22, 14.19, 14.19, 13....","[3.67, 3.67, 3.59, 3.59, 3.68, 3.68, 3.79, 3.7...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.93, 0.93, 0.93, 0.93, 0.93, 0.93, 0.95, 0.9...","[Partly Cloudy, Partly Cloudy, Mostly Cloudy, ..."
4116,MAC000805,2012-04-26,30min,"[0.03, 0.222, 0.067, 0.034, 0.035, 0.079, 0.06...",32304,ToU,ACORN-J,Comfortable,block_73,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[184, 184, 194, 194, 183, 183, 188, 188, 204, ...","[9.82, 9.82, 9.34, 9.34, 9.68, 9.68, 9.32, 9.3...","[7.56, 7.56, 7.17, 7.17, 7.57, 7.57, 7.59, 7.5...","[985.13, 985.13, 985.39, 985.39, 985.45, 985.4...","[6.7, 6.7, 6.32, 6.32, 6.67, 6.67, 5.94, 5.94,...","[7.04, 7.04, 6.28, 6.28, 6.54, 6.54, 7.45, 7.4...","[rain, rain, rain, rain, rain, rain, rain, rai...","[wind, wind, partly-cloudy-night, partly-cloud...","[0.86, 0.86, 0.86, 0.86, 0.87, 0.87, 0.89, 0.8...","[Breezy and Mostly Cloudy, Breezy and Mostly C..."
4118,MAC000846,2012-04-27,30min,"[0.062, 0.062, 0.021, 0.047, 0.021, 0.033, 0.0...",32256,Std,ACORN-J,Comfortable,block_73,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[221, 221, 214, 214, 214, 214, 215, 215, 212, ...","[9.8, 9.8, 9.76, 9.76, 9.78, 9.78, 9.79, 9.79,...","[6.57, 6.57, 6.59, 6.59, 6.66, 6.66, 7.07, 7.0...","[1004.29, 1004.29, 1004.77, 1004.77, 1005.45, ...","[7.29, 7.29, 7.03, 7.03, 7.2, 7.2, 7.34, 7.34,...","[5.13, 5.13, 5.7, 5.7, 5.33, 5.33, 4.95, 4.95,...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.8, 0.8, 0.81, 0.81, 0.81, 0.81, 0.83, 0.83,...","[Mostly Cloudy, Mostly Cloudy, Mostly Cloudy, ..."
4148,MAC003973,2012-07-07,30min,"[0.049, 0.051, 0.095, 0.091, 0.089, 0.057, 0.0...",28848,ToU,ACORN-J,Comfortable,block_73,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[137, 137, 131, 131, 126, 126, 114, 114, 133, ...","[13.96, 13.96, 13.94, 13.94, 13.42, 13.42, 13....","[12.24, 12.24, 12.23, 12.23, 12.06, 12.06, 12....","[1008.32, 1008.32, 1008.09, 1008.09, 1007.93, ...","[13.96, 13.96, 13.94, 13.94, 13.42, 13.42, 13....","[2.03, 2.03, 2.08, 2.08, 2.22, 2.22, 2.28, 2.2...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.89, 0.89, 0.89, 0.89, 0.91, 0.91, 0.94, 0.9...","[Partly Cloudy, Partly Cloudy, Partly Cloudy, ..."


In [9]:
from src.utils.data_utils import compact_to_expanded

In [10]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()

  0%|          | 0/300 [00:00<?, ?it/s]

100%|██████████| 300/300 [00:03<00:00, 87.83it/s]


,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-06-08 00:00:00,MAC001820,0.426,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,199,13.66,9.54,994.85,13.66,8.24,rain,wind,0.76,Breezy and Mostly Cloudy
1,2012-06-08 00:30:00,MAC001820,0.411,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,199,13.66,9.54,994.85,13.66,8.24,rain,wind,0.76,Breezy and Mostly Cloudy
2,2012-06-08 01:00:00,MAC001820,0.337,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,196,13.54,9.89,994.46,13.54,8.27,rain,wind,0.79,Breezy and Mostly Cloudy
3,2012-06-08 01:30:00,MAC001820,0.271,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,196,13.54,9.89,994.46,13.54,8.27,rain,wind,0.79,Breezy and Mostly Cloudy
4,2012-06-08 02:00:00,MAC001820,0.253,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,196,13.15,10.77,994.20,13.15,8.48,rain,wind,0.85,Breezy and Mostly Cloudy


## Reduce Memory Footprint

In [11]:
from src.utils.data_utils import reduce_memory_footprint

In [12]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 9431808 entries, 0 to 36623
Columns: 21 entries, timestamp to summary
dtypes: datetime64[ns](1), float64(8), int64(2), object(10)
memory usage: 6.6 GB


In [13]:
exp_block_df = reduce_memory_footprint(exp_block_df)

In [14]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 9431808 entries, 0 to 36623
Columns: 21 entries, timestamp to summary
dtypes: category(10), datetime64[ns](1), float32(8), int32(2)
memory usage: 602.7 MB


# Train Test Valildation Split

We are going to keep 2014 data as the validation and test period. We have 2 months(Jan and Feb) of data in 2014. Jan is Validation and Feb is Test

In [15]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")

# of Training samples: 8596608 | # of Validation samples: 446400 | # of Test samples: 388800
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [16]:
train.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_train.parquet")
val.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_val.parquet")
test.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_test.parquet")

## Train Test Split after filling in missing values

In [17]:
from src.imputation.interpolation import SeasonalInterpolation

block_df.energy_consumption = block_df.energy_consumption.progress_apply(lambda x: SeasonalInterpolation(seasonal_period=48*7).fit_transform(x.reshape(-1,1)).squeeze())

100%|██████████| 300/300 [00:04<00:00, 60.99it/s]


In [18]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()

100%|██████████| 300/300 [00:03<00:00, 94.19it/s]


,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-06-08 00:00:00,MAC001820,0.426,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,199,13.66,9.54,994.85,13.66,8.24,rain,wind,0.76,Breezy and Mostly Cloudy
1,2012-06-08 00:30:00,MAC001820,0.411,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,199,13.66,9.54,994.85,13.66,8.24,rain,wind,0.76,Breezy and Mostly Cloudy
2,2012-06-08 01:00:00,MAC001820,0.337,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,196,13.54,9.89,994.46,13.54,8.27,rain,wind,0.79,Breezy and Mostly Cloudy
3,2012-06-08 01:30:00,MAC001820,0.271,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,196,13.54,9.89,994.46,13.54,8.27,rain,wind,0.79,Breezy and Mostly Cloudy
4,2012-06-08 02:00:00,MAC001820,0.253,30min,30240,Std,ACORN-J,Comfortable,block_72,NO_HOLIDAY,...,196,13.15,10.77,994.20,13.15,8.48,rain,wind,0.85,Breezy and Mostly Cloudy


## Reduce Memory Footprint

In [19]:
from src.utils.data_utils import reduce_memory_footprint

In [20]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 9431808 entries, 0 to 36623
Columns: 21 entries, timestamp to summary
dtypes: datetime64[ns](1), float64(8), int64(2), object(10)
memory usage: 6.6 GB


In [21]:
exp_block_df = reduce_memory_footprint(exp_block_df)

In [22]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 9431808 entries, 0 to 36623
Columns: 21 entries, timestamp to summary
dtypes: category(10), datetime64[ns](1), float32(8), int32(2)
memory usage: 602.7 MB


In [23]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")

# of Training samples: 8596608 | # of Validation samples: 446400 | # of Test samples: 388800
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [24]:
train.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_train_missing_imputed.parquet")
val.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_val_missing_imputed.parquet")
test.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_test_missing_imputed.parquet")